<a href="https://colab.research.google.com/github/madhurchouhan01/Movie-Recommendation-System/blob/main/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies=movies.merge(credits,on='title')

In [ ]:
movies=movies[['genres', 'overview', 'id', 'keywords', 'title','cast', 'crew']]

In [ ]:
movies.isnull().sum()


genres      0
overview    3
id          0
keywords    0
title       0
cast        0
crew        0
dtype: int64

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
4804    False
4805    False
4806    False
4807    False
4808    False
Length: 4806, dtype: bool

In [ ]:
import ast

In [ ]:
def convert(obj):
  l=[]
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [ ]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
  l=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter!=3:
      l.append(i['name'])
    else:
      break
  return l

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
def find_director(obj):
  l=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      l.append(i['name'])
      break
  return l

In [ ]:
movies['crew']=movies['crew'].apply(find_director)

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['genres']=movies['genres'].apply(lambda x : [i.replace(' ', '') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x : [i.replace(' ', '') for i in x])
movies['cast']=movies['cast'].apply(lambda x : [i.replace(' ', '') for i in x])
movies['crew']=movies['crew'].apply(lambda x : [i.replace(' ', '') for i in x])

In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
new_df=movies[['id','title','tags']]


In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: ''.join(x))

<ipython-input-129-5f53e0fd82e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: ''.join(x))


In [ ]:
new_df.head()

,id,title,tags
0,19995,Avatar,"Inthe22ndcentury,aparaplegicMarineisdispatched..."
1,285,Pirates of the Caribbean: At World's End,"CaptainBarbossa,longbelievedtobedead,hascomeba..."
2,206647,Spectre,AcrypticmessagefromBond’spastsendshimonatrailt...
3,49026,The Dark Knight Rises,"FollowingthedeathofDistrictAttorneyHarveyDent,..."
4,49529,John Carter,"JohnCarterisawar-weary,formermilitarycaptainwh..."


In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

<ipython-input-131-d51011ffdf14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: x.lower())


In [ ]:
new_df.head()

,id,title,tags
0,19995,Avatar,"inthe22ndcentury,aparaplegicmarineisdispatched..."
1,285,Pirates of the Caribbean: At World's End,"captainbarbossa,longbelievedtobedead,hascomeba..."
2,206647,Spectre,acrypticmessagefrombond’spastsendshimonatrailt...
3,49026,The Dark Knight Rises,"followingthedeathofdistrictattorneyharveydent,..."
4,49529,John Carter,"johncarterisawar-weary,formermilitarycaptainwh..."


In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return ''.join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

<ipython-input-136-402dd7cb6b10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)

In [ ]:
def recommend(movie):
  movie_idx=new_df[new_df['title']==movie].index[0]
  distances=similarity[movie_idx]
  movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:6]

  for i in movie_list:
    rec_movies=tuple(new_df.iloc[i[0]].title for i in movie_list)
  return rec_movies


In [ ]:
recommend('Batman Begins')

('Batman v Superman: Dawn of Justice',
 'Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises')

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
textbox = gr.Textbox(label="Enter Movie: ", type="text")
output = gr.Textbox(label="Top 5 Recommended Movies")

In [ ]:
interface=gr.Interface(fn=recommend,inputs=[textbox],outputs=output,title='Movie Recommendation Model')
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e2ec0d8678880c8eaf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
